<a target="_blank" href="https://colab.research.google.com/github/lukebarousse/Int_SQL_Data_Analytics_Course/blob/main/6_Data_Cleaning/1_Handling_Nulls.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Handling Nulls

## Overview

### 🥅 Analysis Goals

- What we’re going to use for this dataset to do X e.g. Use the following in order to explore a dataset on experience and salaries
    - Major topic 1
    - Major topic 2
    - Major topic 3
- The end goal of this is e.g. Identify which jobs meet our expectations of years experience and total salary.

### 📘 Concepts Covered

General concepts we’re going to cover

- Concept 1
- Concept 2
- Concept 3

---
## COALESCE

### 📝 Notes

**`COALESCE()`**

- **COALESCE**: Returns the first non-null value from a list of expressions.

- Syntax:

  ```sql
  SELECT COALESCE(expression1, expression2, ..., default_value);
  ```

- Used to replace `NULL` values with a default. Common in reporting and data cleaning, such as filling missing values with a placeholder.

### 💻 Final Result

- Describe what the final result should be e.g. return the retention by X cohort.

#### Clean Customer Names

**`COALESCE`**

1. Use `COALESCE` on customer's givenname and surname. 

In [ ]:
SELECT
    customerkey,
    COALESCE(givenname, 'Unknown') AS cleaned_givenname,
    COALESCE(surname, 'Unknown') AS cleaned_surname
FROM customer;

<img src="../Resources/query_results/6_handling_nulls_1.png" alt="Query Results 1" style="width: 60%; height: auto;">

#### Cleaned Net Revenue

**`COALESCE`**

1. Write in a query that gets the total net revenue for each customer. 

In [ ]:
    SELECT
        customerkey,
        SUM(quantity * netprice * exchangerate) AS net_revenue
    FROM sales
	GROUP BY
		customerkey

<img src="../Resources/query_results/6_handling_nulls_4.png" alt="Query Results 1" style="width: 50%; height: auto;">

2. Put the query into a CTE (`sales_data`), then `LEFT JOIN` this CTE onto the customer table to return every customer's cleaned name and their net revenue. 

In [ ]:
WITH sales_data AS (
        SELECT
            customerkey,
            SUM(quantity * netprice * exchangerate) AS net_revenue
        FROM sales
        GROUP BY
            customerkey
)

SELECT
    c.customerkey,
    COALESCE(c.givenname, 'Unknown') AS cleaned_givenname,
	COALESCE(c.surname, 'Unknown') AS cleaned_surname,
    COALESCE(s.net_revenue, 0) AS net_revenue
FROM customer c
LEFT JOIN sales_data s ON c.customerkey = s.customerkey;

<img src="../Resources/query_results/6_handling_nulls_5.png" alt="Query Results 1" style="width: 70%; height: auto;">

---
## NULLIF

### 📝 Notes

**`NULLIF`**

- **NULLIF**: Returns `NULL` if two expressions are equal; otherwise, returns the first expression.

- Syntax:

  ```sql
  SELECT NULLIF(expression1, expression2);
  ```

- Helps prevent division by zero by returning `NULL` instead of causing an error.
### 💻 Final Result

- Describe what the final result should be e.g. return the retention by X cohort.

#### Problem Description

**`FUNCTION` / Concept Covered**

1. Get the total number of orders by customer in the `sales_data` CTE and use `COALESCE` on `num_orders` in the main query. 

In [ ]:
WITH sales_data AS (
        SELECT
            customerkey,
            SUM(quantity * netprice * exchangerate) AS net_revenue,
            COUNT(orderkey) AS num_orders
        FROM sales
        GROUP BY
            customerkey
)

SELECT
    c.customerkey,
    COALESCE(c.givenname, 'Unknown') AS cleaned_givenname,
	COALESCE(c.surname, 'Unknown') AS cleaned_surname,
    COALESCE(s.net_revenue, 0) AS net_revenue,
    COALESCE(s.num_orders, 0) AS num_orders -- Added
FROM customer c
LEFT JOIN sales_data s ON c.customerkey = s.customerkey;

<img src="../Resources/query_results/6_handling_nulls_6.png" alt="Query Results 6" style="width: 70%; height: auto;">

2. Return the average order value for each customer by calculating: `net_revenue / num_orders` and add `NULLIF` to `num_orders` to prevent division by zero.

In [ ]:
WITH sales_data AS (
        SELECT
            customerkey,
            SUM(quantity * netprice * exchangerate) AS net_revenue,
            COUNT(orderkey) AS num_orders
        FROM sales
        GROUP BY
            customerkey
)

SELECT
    c.customerkey,
    COALESCE(c.givenname, 'Unknown') AS cleaned_givenname,
	COALESCE(c.surname, 'Unknown') AS cleaned_surname,
    COALESCE(s.net_revenue, 0) AS net_revenue,
    COALESCE(s.num_orders, 0) AS total_orders,
    s.net_revenue / NULLIF(s.num_orders, 0) AS avg_order_value  -- Added: Prevents division by zero
FROM customer c
LEFT JOIN sales_data s ON c.customerkey = s.customerkey;

<img src="../Resources/query_results/6_handling_nulls_7.png" alt="Query Results 7" style="width: 70%; height: auto;">

3. In the `sales_data` find the cohort year by getting the year from the minimum `orderdate` and select `cohort_year` in the main query.

In [ ]:
WITH sales_data AS (
        SELECT
            customerkey,
            EXTRACT(YEAR FROM MIN(orderdate)) AS cohort_year, -- Extract cohort year
            SUM(quantity * netprice * exchangerate) AS net_revenue,
            COUNT(orderkey) AS num_orders
        FROM sales
        GROUP BY
            customerkey
)

SELECT
    c.customerkey,
    c.cohort_year,
    COALESCE(c.givenname, 'Unknown') AS cleaned_givenname,
	COALESCE(c.surname, 'Unknown') AS cleaned_surname,
    COALESCE(s.net_revenue, 0) AS net_revenue,
    COALESCE(s.num_orders, 0) AS total_orders,
    s.net_revenue / NULLIF(s.num_orders, 0) AS avg_order_value  -- Added: Prevents division by zero
FROM customer c
LEFT JOIN sales_data s ON c.customerkey = s.customerkey;

<img src="../Resources/query_results/6_handling_nulls_8.png" alt="Query Results 8" style="width: 90%; height: auto;">